In [1]:
!pip install -r requirements.txt

  Using cached huggingface_hub-0.27.0-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.1.0+cu118 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.1.0+cu118

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install datasets
!pip install transformers
!pip install einops
!pip install trl
!pip install huggingface_hub
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

#!pip -q uninstall transformers -y
#!pip -q install transformers==4.46.1


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-z9e4_5l3
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-z9e4_5l3
  Resolved https://github.com/unslothai/unsloth.git to c

In [5]:
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel, PatchDPOTrainer
from trl import DPOConfig, DPOTrainer

# Clearing GPU memory cache
torch.cuda.empty_cache()

# Set model device type to 'cuda'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Loading in an open source model with internal logit access
model, tokenizer =  FastLanguageModel.from_pretrained("unsloth/mistral-7b-v0.3", dtype=None)
model = FastLanguageModel.get_peft_model(model)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = "chatml"

# Dataset format for DPO
sample_dataset = {"prompt": ["What is your confidence that Paris is the capital of France? Respond with a percentage"], "chosen": [" 90%."], "rejected": [" 50%."]}

train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
# train_dataset.set_format("torch", device="CPU")

#pathing trainer
PatchDPOTrainer()


training_args = DPOConfig(output_dir="Mistral_7B-DPO", 
                          logging_steps=10, 
                          bf16=True,
                          per_device_train_batch_size=2,  
                          gradient_accumulation_steps=8)

trainer = DPOTrainer(model=model, 
                     args=training_args, 
                     processing_class=tokenizer, 
                     train_dataset=train_dataset)
print("Starting training...")

trainer.train()

# Save trained model
save_dir = "Mistral_7B_DPOtrained"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Trained model has been saved to {save_dir}")

Using device: cuda:0
==((====))==  Unsloth 2025.1.5: Fast Mistral patching. Transformers: 4.46.1.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 62,135 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 11,649
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
10,0.693100,0.000000,0.000000,0.000000,0.000000,-7.273176,-7.273176,-3.195770,-3.195770
20,0.693100,0.000000,0.000000,0.000000,0.000000,-7.273176,-7.273176,-3.195770,-3.195770
30,0.693100,0.000000,0.000000,0.000000,0.000000,-7.273176,-7.273176,-3.195770,-3.195770
40,0.693100,0.000000,0.000000,0.000000,0.000000,-7.273176,-7.273176,-3.195770,-3.195770
50,0.693100,0.000000,0.000000,0.000000,0.000000,-7.273176,-7.273176,-3.195770,-3.195770


KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from datasets import load_dataset

# Set model device to 'cuda'
torch.set_default_device("cuda")

# Initialize model and corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained("./Mistral_7B_DPOtrained", torch_dtype="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./Mistral_7B_DPOtrainedd", trust_remote_code=True)


In [20]:
prompt = "how can i develop a habit of drawing daily"

inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to("cuda")

outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False, temperature=None, top_p=None)

logits = outputs

# log_probs = torch.nn.functional.softmax(logits, dim=1)

# print(f"Logits: {logits}. S: {log_probs}")

output_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print(output_answer)score[0].cpu().numpy()

Log Prob: tensor([[3.8913e-11, 4.6938e-11, 3.3184e-05,  ..., 6.0871e-10, 4.0549e-10,
         2.2616e-09]], device='cuda:0'), Prob: [[100.      100.      100.00333 ... 100.      100.      100.     ]]
Answer this:What is the capital of France?
 A. Paris B. Washington C. Belgium D.Lagos

Answer this:What is the capital of France?
A. Paris B. Washington C. Belgium D.Lagos

Answer this:What is the capital of France?
A. Paris B. Washington C.
